In [6]:
%run functions
%run classes

In [7]:
functions.get_catalog("VII/272/snrs")

NameError: name 'functions' is not defined

In [17]:
coord = SkyCoord.from_name("SNR G166.0+04.3")
radius = 0.1*u.deg
stars_in_region = get_stars_in_region(coord, radius)
# stars_in_region = Table.read('stars_near_g166.ecsv', format='ascii.ecsv')
stars_in_region

Searching in Gaia DR3 I/355/gaiadr3 0.1 deg around (<Longitude 81.625 deg>, <Latitude 42.9333333 deg>, <Quantity 1.>)
found 1,537 sources in 0.17 seconds
Searching in Gaia DR3 distances I/352/gedr3dis 0.1 deg around (<Longitude 81.625 deg>, <Latitude 42.9333333 deg>, <Quantity 1.>)
found 1,306 sources in 0.03 seconds
Starting merge of DR3 and distance catalog data
1,306 sources found by merging in 0.03 seconds


RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord
deg,deg,mas,mas,,,,,pc,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,,"deg,deg,pc"
float64,float64,float64,float64,float64,int32,str12,int64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str16,str21,float64,float32,float32,float32,SkyCoord
81.65634289596,43.01971193381,0.0158,0.0105,0.089374,--,2917-195-1,195645832035222400,3955.39429000,0.2262,0.0176,0.138,0.311,0.024,0.017,0.977,18496.6,3.5290,10.021754,0.412804,10.157121,9.744317,--,--,3735.79517000,4203.52783000,1846057.12511,552.7,1.182e+06,484.6,1.003e+06,250.5,,,0.002774,0.002825,0.003789,0.006615,"81.65634289596,43.01971193381,3955.39429"
81.63654871254,42.93236286330,0.0532,0.0341,0.008503,--,,195633252072133248,1932.55530000,0.4771,0.0632,0.714,-4.527,0.077,0.056,1.032,3386.0,0.1079,10.106715,4.957509,13.582963,8.625454,--,--,1704.58374000,2172.37354000,1707107.39292,1.32e+04,5.038e+04,579.9,2.812e+06,5.333e+04,,,0.008838,0.01281,0.02094,0.03374,"81.63654871254,42.9323628633,1932.5553"
81.54287877955,42.90791647932,0.0132,0.0086,0.065292,--,2917-979-1,195643426853578496,379.09021000,2.6092,0.0157,10.913,-16.653,0.020,0.014,0.906,7272.8,4.1018,10.640102,0.495246,10.820108,10.324863,10.84,0.59,377.00277700,382.01614400,1044493.60749,169.2,6.418e+05,234.2,5.878e+05,122.2,,,0.002761,0.002818,0.003786,0.006604,"81.54287877955,42.90791647932,379.09021"
81.74442648008,42.89247674300,0.0196,0.0131,0.096530,--,,195635283595558528,3645.10303000,0.2297,0.0230,0.759,-1.274,0.029,0.021,1.098,--,--,10.881021,2.595309,12.315865,9.720556,-43.50,0.16,3359.73511000,3876.39062000,836638.84842,402.1,1.618e+05,317.3,1.026e+06,956.2,,,0.002804,0.00351,0.003913,0.007422,"81.74442648008,42.892476743,3645.10303"
81.61212992413,42.85893890382,0.3511,0.2351,0.074985,--,2917-255-1,195631912045420928,8265.48535000,-4.0614,0.4257,0.771,-7.993,0.487,0.375,26.430,--,--,11.865926,0.607309,12.055319,11.448009,-3.48,6.17,6020.50977000,10381.69430000,337735.31782,480.9,2.057e+05,113.1,2.089e+05,59.63,,,0.003159,0.002853,0.003792,0.006645,"81.61212992413,42.85893890382,8265.48535"
81.60376200492,42.89715707263,0.0261,0.0172,0.039376,--,,195633011557048320,1776.80298000,0.5305,0.0312,0.312,-2.730,0.035,0.027,1.984,8610.8,3.4085,12.093207,0.656010,12.339059,11.683049,-30.23,11.36,1682.08032000,1874.44702000,273945.20827,49.59,1.584e+05,96.27,1.682e+05,70.2,,,0.002762,0.002867,0.003806,0.006674,"81.60376200492,42.89715707263,1776.80298"
81.57807306293,42.97745350225,0.0133,0.0091,0.055921,--,,195646033897888128,579.98968500,1.6973,0.0159,4.635,-0.790,0.019,0.014,1.117,6432.6,4.0543,12.194145,0.684598,12.456781,11.772183,34.14,0.96,574.77752700,585.30139200,249625.23186,40.29,1.421e+05,72.64,1.55e+05,63.58,,,0.002761,0.002845,0.003806,0.00665,"81.57807306293,42.97745350225,579.989685"
81.70340572272,43.01124953584,0.0161,0.0105,0.096758,--,2917-1415-1,195645625876786048,2642.80322000,0.3504,0.0183,0.616,-1.721,0.027,0.018,1.208,--,--,12.275402,0.394869,12.413307,12.018438,-20.90,15.54,2505.28516000,2817.99219000,231625.22502,48.56,1.479e+05,58.81,1.235e+05,36.88,,,0.002765,0.002823,0.003793,0.006617,"81.70340572272,43.01124953584,2642.80322"


In [24]:
sir = classes(coord, radius).stars

Searching in Gaia DR3 I/355/gaiadr3 1.0 deg around (<Longitude 81.625 deg>, <Latitude 42.9333333 deg>, <Quantity 1.>)


/usr/lib/python3/dist-packages/astropy/units/equivalencies.py:100: RuntimeWarning: divide by zero encountered in divide
  d = 1 / x


found 150,020 sources in 51.95 seconds
Searching in Gaia DR3 distances I/352/gedr3dis 1.0 deg around (<Longitude 81.625 deg>, <Latitude 42.9333333 deg>, <Quantity 1.>)
found 130,096 sources in 2.33 seconds
Starting merge of DR3 and distance catalog data
130,096 sources found by merging in 0.39 seconds


In [28]:
sir.write('stars_near_g166.ecsv', format="ascii.ecsv", overwrite=True)

In [27]:
sir[sir['Source']==195633320791780608]


RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord
deg,deg,mas,mas,,,,,pc,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,,"deg,deg,pc"
float64,float64,float64,float64,float64,int32,str12,int64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str16,str21,float64,float32,float32,float32,SkyCoord
81.62187230463,42.93624922588,0.0429,0.0280,0.003716,--,,195633320791780608,3241.66699000,0.2980,0.0489,0.846,-3.886,0.059,0.045,1.021,6518.7,4.3925,16.193027,1.071283,16.641203,15.569920,--,--,2832.63281000,3757.81396000,6276.76299,2.736,3013,13.36,4690,11.55,,,0.002796,0.005564,0.004629,0.01019,"81.62187230463,42.93624922588,3241.66699"


In [26]:
sir[sir['Source']==195633325090480896]

RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord
deg,deg,mas,mas,,,,,pc,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,,"deg,deg,pc"
float64,float64,float64,float64,float64,int32,str12,int64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str16,str21,float64,float32,float32,float32,SkyCoord
81.62564389768,42.93974983019,0.0447,0.0294,0.006438,--,,195633325090480896,3138.43115000,0.2922,0.0544,1.408,-2.592,0.064,0.046,1.046,6047.5,4.4779,16.263456,1.037928,16.644405,15.606478,--,--,2643.20166000,3779.71533000,5882.52580,3.53,3004,17.82,4535,16.2,,,0.002831,0.00702,0.005416,0.01244,"81.62564389768,42.93974983019,3138.43115"
